# Data Preprocessing

In [331]:
# Install the surprise package
!pip install -q -U scikit-surprise
from surprise import Dataset, Reader
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import arff
import re

## Loading Data

In [2]:
# Reading and processing the data in chunks for large datasets or busy RAM
def read_and_process_json_in_chunks(path, chunksize=10000, dtype=None):
    chunks = []
    for chunk in pd.read_json(path, lines=True, dtype=dtype, chunksize=chunksize):
        chunks.append(chunk)
    return pd.concat(chunks, ignore_index=True)

In [3]:
path_ratings = os.path.expanduser('../data/Kindle_Store_5.json.gz')
ratings = read_and_process_json_in_chunks(path_ratings, dtype= {'reviewerID': str, 'asin': str, 'overall': int, 'reviewTime': str})

In [14]:
path_meta = os.path.expanduser('../data/meta_Kindle_Store.json.gz')
meta_data = read_and_process_json_in_chunks(path_meta,  dtype= {'category': str})

In [5]:
preprocessing_ratings = ratings.copy()

In [15]:
preprocessing_meta = meta_data.copy()

## Preprocessing Ratings

In [7]:
# change the name of the ratings column from overall to rating to make this more clear
preprocessing_ratings.rename(columns={'overall':'rating'}, inplace=True)

In [43]:
# Drop Duplicate Rating entries
preprocessing_ratings.drop_duplicates(subset=['reviewerID', 'asin'], keep='first', inplace=True, ignore_index=False)

In [8]:
# Filter out every style except Kindle (Paperback, Audible, Hardcover, MP3 CD remove)
if 'style' in preprocessing_ratings.columns:
    # Filter the DataFrame to include only rows where 'style' column contains only Kindle
    preprocessing_ratings = preprocessing_ratings[preprocessing_ratings['style'].astype(str).str.contains("{'Format:': ' Kindle Edition'}")]
    # Remove verified column, as we assume, all remaining ratings are for Products within the Kindle Edition
    preprocessing_ratings = preprocessing_ratings.drop(columns=['style'])

In [9]:
# Filter by verified and remove unverified reviews
if 'verified' in preprocessing_ratings.columns:
    preprocessing_ratings = preprocessing_ratings[preprocessing_ratings['verified'] == True]
    # Remove verified column, as we assume, all remaining ratings are verified
    preprocessing_ratings.drop(columns=['verified'], inplace=True)

# Drop unixReviewTime
if 'unixReviewTime' in preprocessing_ratings.columns:
    preprocessing_ratings.drop(columns=['unixReviewTime'], inplace=True)

# Drop unixReviewTime
if 'reviewTime' in preprocessing_ratings.columns:
    preprocessing_ratings.drop(columns=['reviewTime'], inplace=True)

# Drop the 'image' column
if 'image' in preprocessing_ratings.columns:
    preprocessing_ratings = preprocessing_ratings.drop(columns=['image'])

# Drop the 'reviewText' column
if 'reviewText' in preprocessing_ratings.columns:
    preprocessing_ratings = preprocessing_ratings.drop(columns=['reviewText'])

# Drop the 'summary' column
if 'summary' in preprocessing_ratings.columns:
    preprocessing_ratings = preprocessing_ratings.drop(columns=['summary'])

# Drop the 'reviewerName' column
if 'reviewerName' in preprocessing_ratings.columns:
    preprocessing_ratings = preprocessing_ratings.drop(columns=['reviewerName'])

# Drop the 'vote' column
if 'vote' in preprocessing_ratings.columns:
    preprocessing_ratings = preprocessing_ratings.drop(columns=['vote'])

In [11]:
preprocessing_ratings.head()

,rating,reviewerID,asin
0,4,A2LSKD2H9U8N0J,B000FA5KK0
1,5,A2QP13XTJND1QS,B000FA5KK0
2,5,A8WQ7MAG3HFOZ,B000FA5KK0
3,5,A1E0MODSRYP7O,B000FA5KK0
4,5,AYUTCGVSM1H7T,B000FA5KK0


## Preprocessing Meta Data

In [47]:
# Drop Duplicate Rating entries
preprocessing_meta.drop_duplicates(subset=['title', 'asin'], keep='first', inplace=True, ignore_index=False)

In [16]:
# Replace '&#39;' with '&' and delete rows where 'title' is an empty string, as this is necessary for the content-based filtering
preprocessing_meta['title'] = preprocessing_meta['title'].str.replace('&#39;', '&')
preprocessing_meta['title'] = preprocessing_meta['title'].str.replace('&amp;', '&')
preprocessing_meta['title'] = preprocessing_meta['title'].str.replace('&s;', "'s")
preprocessing_meta = preprocessing_meta[preprocessing_meta['title'] != '']

In [17]:
# #Extract the book language, print length and Publication year and turn it in a new column each
if 'details' in preprocessing_meta.columns:

    # Add the Languages of single books to a new column
    languages = []
    for index, row in preprocessing_meta.iterrows():
        # Extract the 'Language' value from the 'details' dictionary
        language = row['details'].get('Language:', None)
        # Append the extracted value to the 'languages' list
        languages.append(language)
    
    # Add the 'languages' list as a new column 'Language' to the DataFrame
    preprocessing_meta['language'] = languages

    
    # Create an empty list to store extracted print lengths
    print_lengths = []
    
    # Iterate over each row in the DataFrame
    for index, row in preprocessing_meta.iterrows():
        # Extract the 'Print Length' value from the 'details' dictionary
        print_length_str = row['details'].get('Print Length:', None)
        # Extracting only the numeric part from the string
        if print_length_str:
            print_length = ''.join(filter(str.isdigit, print_length_str))
        else:
            print_length = None
        # Convert the extracted value to an integer
        if print_length:
            try:
                print_length = int(print_length)
            except ValueError:
                print_length = None  # Handle non-finite values
        # Append the extracted value to the 'print_lengths' list
        print_lengths.append(print_length)
    
    # Add the 'print_lengths' list as a new column 'Print_Length' to the DataFrame
    preprocessing_meta['print_length'] = print_lengths
    
    # Convert the 'Print_Length' column to integers, handling non-finite values
    preprocessing_meta['print_length'] = preprocessing_meta['print_length'].astype('Int64')
    
    # Define bins and labels for categories
    bins = [0, 300, 500, 700, 1000, float('inf')]
    labels = ['small', 'medium', 'large', 'massive']
    
    # Fill NaN values with a placeholder value (-1 in this case)
    preprocessing_meta['print_length'] = preprocessing_meta['print_length'].fillna(-1)
    
    # Create a new column 'Print_Length_Category' based on the bins
    preprocessing_meta['print_length_category'] = np.digitize(preprocessing_meta['print_length'], bins=bins, right=False)
    
    # Map values over 1000 to 'massive'
    preprocessing_meta['print_length_category'] = np.where(preprocessing_meta['print_length'] > 1000, len(labels), preprocessing_meta['print_length_category'])
    
    # Map bin indices to labels
    preprocessing_meta['print_length_category'] = preprocessing_meta['print_length_category'].map({i: l for i, l in enumerate(labels, 1)})

    #fill nan values with medium
    preprocessing_meta['print_length_category'].fillna('medium', inplace=True)

    preprocessing_meta = preprocessing_meta.drop(columns=['print_length'])
    

    # Create an empty list to store extracted publication years
    publication_years = []
    
    # Iterate over each row in the DataFrame
    for index, row in preprocessing_meta.iterrows():
        # Extract the 'Publication Date' value from the 'details' dictionary
        publication_date_str = row['details'].get('Publication Date:', None)
        
        # Extracting only the year part from the string
        if publication_date_str:
            publication_year_str = publication_date_str.split()[-1]
            
            # Convert the year string to an integer
            try:
                publication_year = publication_year_str
            except ValueError:
                publication_year = None  # Handle cases where conversion to int fails
        else:
            publication_year = None
        
        # Append the extracted value to the 'publication_years' list
        publication_years.append(publication_year)
    
    # Add the 'publication_years' list as a new column 'publication_year' to the DataFrame
    preprocessing_meta['publication_year'] = publication_years
    
    # Delete the 'details' column
    preprocessing_meta.drop(columns=['details'], inplace=True)


In [18]:
def process_category_string(category_string):
    # Function to clean and extract categories from the category string
    
    # Use regular expression to find categories within square brackets
    categories = re.findall(r"'(.*?)'", category_string)

    # Join the categories with comma
    cleaned_categories = ', '.join(categories)

    # Remove </span> and everything after it
    cleaned_categories = re.sub(r', </span>', '', cleaned_categories)
    
    # Remove HTML tags, keeping only the text
    cleaned_categories = re.sub(r'<a class="a-link-normal" href="[^"]*">([^<]*)</a>', r'\1', cleaned_categories)
    
    return cleaned_categories

preprocessing_meta['category_string'] = preprocessing_meta['category'].apply(process_category_string)


In [24]:
# Delete unused columns

if 'tech1' in preprocessing_meta.columns:
    preprocessing_meta.drop(columns=['tech1'], inplace=True)

if 'tech2' in preprocessing_meta.columns:
    preprocessing_meta.drop(columns=['tech2'], inplace=True)

if 'description' in preprocessing_meta.columns:
    preprocessing_meta.drop(columns=['description'], inplace=True)

if 'fit' in preprocessing_meta.columns:
    preprocessing_meta.drop(columns=['fit'], inplace=True)

if 'feature' in preprocessing_meta.columns:
    preprocessing_meta.drop(columns=['feature'], inplace=True)

if 'main_cat' in preprocessing_meta.columns:
    preprocessing_meta.drop(columns=['main_cat'], inplace=True)

if 'similar_item' in preprocessing_meta.columns:
    preprocessing_meta.drop(columns=['similar_item'], inplace=True)

if 'imageURL' in preprocessing_meta.columns:
    preprocessing_meta.drop(columns=['imageURL'], inplace=True)

if 'imageURLHighRes' in preprocessing_meta.columns:
    preprocessing_meta.drop(columns=['imageURLHighRes'], inplace=True)

if 'also_buy' in preprocessing_meta.columns:
    preprocessing_meta.drop(columns=['also_buy'], inplace=True)

if 'also_view' in preprocessing_meta.columns:
    preprocessing_meta.drop(columns=['also_view'], inplace=True)

if 'date' in preprocessing_meta.columns:
    preprocessing_meta.drop(columns=['date'], inplace=True)

if 'price' in preprocessing_meta.columns:
    preprocessing_meta.drop(columns=['price'], inplace=True)

if 'category' in preprocessing_meta.columns:
    #delete original category columns
    preprocessing_meta = preprocessing_meta.drop(columns=['category'])

In [23]:
# Create new column paid_free and drop the 'rank' column
if 'rank' in preprocessing_meta.columns:

    # Create a new column 'paid_free'
    preprocessing_meta['paid_free'] = preprocessing_meta['rank'].apply(lambda x: 'Paid' if 'Paid' in str(x) else 'Free')

    # Drop the 'rank' column
    preprocessing_meta.drop(columns=['rank'], inplace=True)

In [123]:
#preprare the textfield 'book_info' for content-based models, as we don't have product description
preprocessing_meta["book_info"] =  preprocessing_meta['category_string'] + '  ' + preprocessing_meta['brand'] + '  ' + preprocessing_meta['paid_free']+ ' ' + preprocessing_meta['print_length_category'] + ' ' + preprocessing_meta['publication_year'] + '  ' + preprocessing_meta['language'] 

# Print count of nan
nan_count = preprocessing_meta["book_info"].isna().sum()
print(f'The number of NaN values in "book_info" column: {nan_count}')

## Merge Data
Finally the ratings and meta data are merged to have on consistent dataset

In [640]:
data = pd.merge(preprocessing_ratings, preprocessing_meta, how="inner", on=["asin"])

In [641]:
len(data)

1361844

In [642]:
sample_size = 1000000  # Adjust the sample size as needed - 1100000 for 46922
data = data.sample(n=sample_size, random_state=42)

In [643]:
# Keep only items with at least a certain number of reviews - 51 für 46922
min_reviews_per_item = 51
popular_items = data['asin'].value_counts()
popular_items = popular_items[popular_items >= min_reviews_per_item].index

df_popular_items = data[data['asin'].isin(popular_items)]


In [644]:
len(df_popular_items)

259219

In [645]:
# Keep only users with at least a certain number of reviews - 22 für 46922
min_reviews_per_user = 22
active_users = df_popular_items['reviewerID'].value_counts()
active_users = active_users[active_users >= min_reviews_per_user].index

df_active_users = df_popular_items[df_popular_items['reviewerID'].isin(active_users)]


In [646]:
len(df_active_users)

33036

In [647]:
# new is 20-250 with 220.000 sample
# not new is 56-200 with 500.000 sample
df_active_users.to_excel("../data/data_kindle_preprocessed_smaller.xlsx", sheet_name='Data')  

In [622]:
df_active_users.columns

Index(['rating', 'reviewerID', 'asin', 'title', 'brand', 'language',
       'print_length_category', 'publication_year', 'category_string',
       'paid_free', 'book_info'],
      dtype='object')